<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Silvia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import csv

'''Provides a high-performance multidimensional array object, 
and tools for working with these arrays'''
import numpy as np
'''Save an array to a text file.'''
from numpy import savetxt

'''Providing high-performance, easy-to-use data structures and data analysis tools'''
import pandas as pd
'''Two-dimensional size-mutable, 
potentially heterogeneous tabular data structure with labeled axes (rows and columns). 
Arithmetic operations align on both row and column labels.'''
from pandas import DataFrame
'''Read CSV file into DataFrame.
Supports optionally iterating or breaking of the file into chunks.'''
from pandas import read_csv
'''Concatenate pandas objects along a particular axis with optional set logic along the other axes.
Can also add a layer of hierarchical indexing on the concatenation axis, 
which may be useful if the labels are the same (or overlapping) on the passed axis number'''
from pandas import concat

from keras import optimizers
'''Linear stack of layers.'''
from keras.models import Sequential
'''Loads a model saved via save_model.'''
from keras.models import load_model
'''Dense: just your regular densely-connected NN layer.
Dropout: applies Dropout to the input.'''
from keras.layers import Dense, Dropout, BatchNormalization,Activation
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model(shape):
  
  model = Sequential()
 
  '''model.add(Dense(113, input_dim=shape, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))'''
  model.add(Dense(113, input_dim=shape, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  '''model.add(Dense(229, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))'''
  model.add(Dense(229, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  '''model.add(Dense(153, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))'''
  model.add(Dense(153, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  '''model.add(Dense(1, activation='sigmoid'))'''
  model.add(Dense(1))
  model.add(BatchNormalization())
  model.add(Activation('sigmoid'))
  
  #print(model.summary())

  return model

In [20]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/6_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

DATASET LOADED
Converting values...
CONVERSION DONE
Loading data...


In [21]:
#OPTIMIZERS
sgd = optimizers.SGD(lr=0.0001)
adm = optimizers.Adam(lr=0.0001)
ada = optimizers.Adadelta(lr=0.0001)
#rms = optimizers.RMSprop(lr=0.001)

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = KFold(n_splits=10, shuffle=False, random_state=None)
cvscores = []
predictions = []

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])
  
  #EARLY STOPPING
  es = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=0, mode='auto')

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=5, validation_split=0.33, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight, callbacks = [es])

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 1)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  pred = model.predict_classes(X[test], batch_size=128, verbose=1)
  predictions.append(pred)
  
  if(i==10): predictions.append(Y[test])
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



FOLD: 1
Train on 138647 samples, validate on 68290 samples
Epoch 1/5
138647/138647 [==============================] - 12s 83us/step - loss: 0.4963 - acc: 0.6014 - val_loss: 0.5500 - val_acc: 0.6209
Epoch 2/5
138647/138647 [==============================] - 9s 68us/step - loss: 0.4385 - acc: 0.6796 - val_loss: 0.5259 - val_acc: 0.7082
Epoch 3/5
138647/138647 [==============================] - 9s 67us/step - loss: 0.4087 - acc: 0.7197 - val_loss: 0.5402 - val_acc: 0.7031
Epoch 4/5
22993/22993 [==============================] - 0s 21us/step
Test loss: 46.04%
Test accuracy: 84.45%
22993/22993 [==============================] - 1s 43us/step

FOLD: 2
Train on 138647 samples, validate on 68290 samples
Epoch 1/5
138647/138647 [==============================] - 12s 85us/step - loss: 0.4815 - acc: 0.6408 - val_loss: 0.5374 - val_acc: 0.6087
Epoch 2/5
138647/138647 [==============================] - 10s 70us/step - loss: 0.4232 - acc: 0.7094 - val_loss: 0.5115 - val_acc: 0.6862
Epoch 3/5
138647/

In [0]:
#model.save('my_model1_cv.h5')

#model = load_model('my_model1_cv.h5')